In [ ]:
!pip install umap-learn[plot]

In [1]:
import json
import requests
import pandas as pd

from operator import itemgetter
from sklearn.cluster import SpectralClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
url = 'http://tools.kinds.or.kr:8888/search/news'

datas = {
    "access_key": "9af2f705-2974-4340-8b55-d69040b944ab",

    "argument": {
        "query": {"title": ""},
        "published_at": {
            "from": "2021-09-02",
            "until": "2021-10-11"
        },
        "provider": [],
        "category": ["008004000"],
        "category_incident": [""],
        "byline": "",
        "provider_subject": [""],
        "subject_info": [""],
        "subject_info1": [""],
        "subject_info2": [""],
        "subject_info3": [""],
        "subject_info4": [""],
        "sort": {"date": "asc"},
        "hilight": 200,
        "return_from": 0,
        "return_size": 10000,
        "fields": [
            "byline",
            "category",
            "category_incident",
            "provider_news_id",
            "hilight",
        ]
    }
}

response = requests.post(url, data=json.dumps(datas))
res = response.json()

In [3]:
total_hits = res['return_object']['total_hits']
print("결과 개수:",total_hits)

결과 개수: 9658


In [4]:
# api데이터로 코퍼스 생성
def create_corpus(total_hits):
    global news_ids
    news_ids, titles, hilights ,title_plus_hilights = [], [], [],[]

    sentence_tokenized_text = []

    for i in range(total_hits):
        title = res['return_object']['documents'][i]['title']
        hilight = res['return_object']['documents'][i]['hilight']
        news_id = res['return_object']['documents'][i]['news_id']
        news_ids.append(news_id)
        title_plus_hilight = title +","+ hilight
        title_plus_hilights.append(title_plus_hilight)
    len(title_plus_hilights)

    return title_plus_hilights

In [5]:
count_vect = CountVectorizer()
DTM = count_vect.fit_transform(create_corpus(total_hits))
DTM.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
tdm=DTM.T

In [ ]:
spc = SpectralClustering(affinity=cosine_similarity,random_state=41)
labels = spc.fit_predict(tdm)
labels

In [15]:
words = count_vect.get_feature_names()

def top10(labels):
    freq_words = []
    for i in range(4):
        count = tdm[labels == i, :].sum(axis=0)
        ws = [w for w, n in sorted(zip(words, count.flat), key=itemgetter(1), reverse=True)[:10]]
        freq_words.append(ws)

    return pd.DataFrame(freq_words)

In [16]:
top10(labels)

,0,1,2,3,4,5,6,7,8,9
0,10위권,14회째를,1980,1조1000억,20주년,350명이,360도,45만명이,4사는,4성
1,wǒmen,4회차를,6개사의,kbo,m22,00,0001초,000210,000gb의,000개
2,4300여건을,00,0001초,000210,000gb의,000개,000건을,000대,000대를,000만
3,agba는,bgf교육연구센터를,gkl에,nhn페이코가,00,0001초,000210,000gb의,000개,000건을
